In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from warnings import simplefilter

simplefilter('ignore')

In [53]:
data=data.drop(index = list(range(data.shape[0] // 16, data.shape[0])))

In [51]:
data = pd.read_csv('Hotel_Reviews.csv')

In [54]:
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Hotel_Address                               2014 non-null   object 
 1   Additional_Number_of_Scoring                2014 non-null   int64  
 2   Review_Date                                 2014 non-null   object 
 3   Average_Score                               2014 non-null   float64
 4   Hotel_Name                                  2014 non-null   object 
 5   Reviewer_Nationality                        2014 non-null   object 
 6   Negative_Review                             2014 non-null   object 
 7   Review_Total_Negative_Word_Counts           2014 non-null   int64  
 8   Total_Number_of_Reviews                     2014 non-null   int64  
 9   Positive_Review                             2014 non-null   object 
 10  Review_Total

In [56]:
category_cols = ['Hotel_Address','Review_Date', 'Hotel_Name', 'Reviewer_Nationality', 'Negative_Review','Positive_Review','Tags','days_since_review']

print('Количество уникальных значений\n')
for col in category_cols:
    print(f'{col}: {data[col].unique().size}')

Количество уникальных значений

Hotel_Address: 4
Review_Date: 623
Hotel_Name: 4
Reviewer_Nationality: 76
Negative_Review: 1284
Positive_Review: 1889
Tags: 567
days_since_review: 623


In [57]:
remove_cols = ['Positive_Review']

In [58]:
for col in remove_cols:
    category_cols.remove(col)
data = pd.get_dummies(data, columns=category_cols)

In [59]:
data.drop(remove_cols, axis=1, inplace=True)
data.describe()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,lat,lng,Hotel_Address_ s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,...,days_since_review_88 days,days_since_review_90 days,days_since_review_91 days,days_since_review_92 days,days_since_review_93 days,days_since_review_94 days,days_since_review_95 days,days_since_review_97 days,days_since_review_98 days,days_since_review_99 days
count,2014.00000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,...,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000
mean,439.75422,8.697170,17.711023,2158.264151,20.815293,7.040715,8.804320,51.677889,0.877236,0.201092,...,0.000993,0.000993,0.001986,0.000497,0.000993,0.001490,0.000497,0.001490,0.000993,0.000993
std,210.98743,0.586685,30.789510,518.600748,21.988270,10.363792,1.435705,0.342720,2.027095,0.400916,...,0.031505,0.031505,0.044532,0.022283,0.031505,0.038576,0.022283,0.038576,0.031505,0.031505
min,194.00000,7.700000,0.000000,1403.000000,0.000000,1.000000,2.500000,51.491888,-0.194971,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,244.00000,8.500000,0.000000,1831.000000,7.000000,1.000000,8.300000,51.491888,-0.194971,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,639.00000,9.200000,7.000000,2619.000000,14.000000,3.000000,9.200000,51.513734,-0.108751,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,639.00000,9.200000,22.000000,2619.000000,26.000000,8.000000,10.000000,51.513734,-0.108751,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1058.00000,9.200000,397.000000,4380.000000,186.000000,85.000000,10.000000,52.360576,4.915968,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [60]:

y = data['Average_Score']
X = data.drop('Average_Score', axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

In [61]:
scaler = MinMaxScaler().fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_train.columns)
x_train.describe()

,Additional_Number_of_Scoring,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,lat,lng,Hotel_Address_ s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,Hotel_Address_1 15 Templeton Place Earl s Court Kensington and Chelsea London SW5 9NB United Kingdom,...,days_since_review_88 days,days_since_review_90 days,days_since_review_91 days,days_since_review_92 days,days_since_review_93 days,days_since_review_94 days,days_since_review_95 days,days_since_review_97 days,days_since_review_98 days,days_since_review_99 days
count,1409.000000,1409.000000,1409.000000,1409.000000,1409.000000,1409.000000,1409.000000,1409.000000,1409.000000,1409.000000,...,1409.000000,1409.000000,1409.000000,1409.0,1409.000000,1409.0,1409.000000,1409.000000,1409.000000,1409.000000
mean,0.287270,0.044539,0.256482,0.114304,0.074270,0.843066,0.205473,0.201097,0.192335,0.285309,...,0.000710,0.000710,0.001419,0.0,0.000710,0.0,0.000710,0.001419,0.001419,0.001419
std,0.244230,0.078297,0.173908,0.122850,0.125505,0.188319,0.388012,0.390065,0.394275,0.451722,...,0.026641,0.026641,0.037662,0.0,0.026641,0.0,0.026641,0.037662,0.037662,0.037662
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.057870,0.000000,0.143769,0.037634,0.000000,0.773333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,0.515046,0.017632,0.408465,0.075269,0.024390,0.893333,0.025148,0.016870,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,0.515046,0.052897,0.408465,0.145161,0.085366,1.000000,0.025148,0.016870,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.0,1.000000,0.0,1.000000,1.000000,1.000000,1.000000


In [62]:
def print_metrics(y_test, y_pred):
    print(f"R^2: {r2_score(y_test, y_pred)}")
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
    
def print_cv_result(cv_model, x_test, y_test):
    print(f'Оптимизация метрики {cv_model.scoring}: {cv_model.best_score_}')
    print(f'Лучший параметр: {cv_model.best_params_}')
    print('Метрики на тестовом наборе')
    print_metrics(y_test, cv_model.predict(x_test))
    print()

In [63]:
base_k = 6
base_knn = KNeighborsRegressor(n_neighbors=base_k)
base_knn.fit(x_train, y_train)
y_pred_base = base_knn.predict(x_test)
print(f'Test metrics for KNN with k={base_k}\n')
print_metrics(y_test, y_pred_base)

Test metrics for KNN with k=6

R^2: 0.9999183374131726
MSE: 2.9384756657483724e-05
MAE: 0.00022038567493202715


In [64]:
metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
cv_values = [5, 10]

for cv in cv_values:
    print(f'Результаты кросс-валидации при cv={cv}\n')
    for metric in metrics:    
        params = {'n_neighbors': range(1, 30)}
        knn_cv = GridSearchCV(KNeighborsRegressor(), params, cv=cv, scoring=metric, n_jobs=-1)
        knn_cv.fit(x_train, y_train)
        print_cv_result(knn_cv, x_test, y_test)

Результаты кросс-валидации при cv=5

Оптимизация метрики r2: 1.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Оптимизация метрики neg_mean_squared_error: 0.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Оптимизация метрики neg_mean_absolute_error: 0.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Результаты кросс-валидации при cv=10

Оптимизация метрики r2: 1.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Оптимизация метрики neg_mean_squared_error: 0.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Оптимизация метрики neg_mean_absolute_error: 0.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0



In [65]:
metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

print('Результаты кросс-валидации\n')
for metric in metrics:
    params = {'n_neighbors': range(1, 30)}
    knn_cv = GridSearchCV(KNeighborsRegressor(), params, cv=5, scoring=metric, n_jobs=-1)
    knn_cv.fit(x_train, y_train)
    print_cv_result(knn_cv, x_test, y_test)

Результаты кросс-валидации

Оптимизация метрики r2: 1.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Оптимизация метрики neg_mean_squared_error: 0.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0

Оптимизация метрики neg_mean_absolute_error: 0.0
Лучший параметр: {'n_neighbors': 1}
Метрики на тестовом наборе
R^2: 1.0
MSE: 0.0
MAE: 0.0



In [66]:
best_k = 2
y_pred_best = KNeighborsRegressor(n_neighbors=best_k).fit(x_train, y_train).predict(x_test)

In [67]:
print('Basic model\n')
print_metrics(y_test, y_pred_base)
print('_______________________')
print('\nOptimal model\n')
print_metrics(y_test, y_pred_best)

Basic model

R^2: 0.9999183374131726
MSE: 2.9384756657483724e-05
MAE: 0.00022038567493202715
_______________________

Optimal model

R^2: 1.0
MSE: 0.0
MAE: 0.0
